[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/enterprise/healthcare/ChunkMergeClinical.ipynb)

In [4]:
import json

with open('keys.json') as f:
    license_keys = json.load(f)

license_keys.keys()


dict_keys(['secret', 'SPARK_NLP_LICENSE', 'JSL_OCR_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_OCR_SECRET'])

In [4]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

secret = license_keys['secret']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['JSL_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']

! python -m pip install --upgrade spark-nlp-jsl==2.5.2  --extra-index-url https://pypi.johnsnowlabs.com/$secret

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl



def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.0") \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-2.5.0.jar")
      
    return builder.getOrCreate()


spark = start(secret) # if you want to start the session with custom params as in start function above
# sparknlp_jsl.start(secret)

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/l5rISdi5Xk
     |████████████████████████████████| 122kB 7.8MB/s 
2.5.0


In [1]:


import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

In [2]:
data_chunk_merge = spark.createDataFrame([
  (1,"""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to 
presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis 
three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index 
( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting.
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . 
She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was 
significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , 
or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , 
anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin 
( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed 
as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior 
to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , 
the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , 
and lipase was 52 U/L .
 β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged 
 and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
 The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides 
 to 1400 mg/dL , within 24 hours .
 Twenty days ago.
 Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
 At birth the typical boy is growing slightly faster than the typical girl, but the velocities become equal at about 
 seven months, and then the girl grows faster until four years. 
 From then until adolescence no differences in velocity 
 can be detected. 21-02-2020 
21/04/2020
""")]).toDF("id","text")


regex = '''(c|p|yc|yp|r|rp|a)?(C[1-5])?M(x|X|0|1[a-d]?),pM
(c|p|yc|yp|r|rp|a)?(C[1-5])?N(x|X|0|[1-3][a-d]?),pN
(c|p|yc|yp|r|rp|a)?(C[1-5])?T(x|X|is|0|[1-4][a-d]?),pT
([0-9]+(\.[0-9]+)?\s?x\s?)*([0-9]+(\.[0-9]+)?)\s?(mg|MG|mm|cm|MM|CM|),SIZE
at Los Angeles California,LOCATION
Zacarias,PERSON
better than,BLOCK'''

with open('ner_regex.csv', 'w') as f:
    f.write(regex)

replace_dict = '''pT,TNM
pM,TNM'''

with open('replace_dict.csv', 'w') as f:
    f.write(replace_dict)

false_positives = '''beautiful thing,BLOCK'''

with open('false_positives.csv', 'w') as f:
    f.write(false_positives)

In [3]:
gender = '''male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,neutral'''

with open('gender.csv', 'w') as f:
    f.write(gender)


gender = {
  "entity": "Gender",
  "ruleScope": "sentence", 
  "completeMatchRegex": "true"
}

import json

with open('gender.json', 'w') as f:
    json.dump(gender, f)


date = {
  "entity": "Date ",
  "ruleScope": "sentence",
  "regex": "\\d{1,2}[\\/\\-\\:]{1}(\\d{1,2}[\\/\\-\\:]{1}){0,1}\\d{2,4}",
  "valuesDefinition":[],
  "prefix": [],
  "suffix": [],
  "contextLength": 150,
  "context": []
}

with open('date.json', 'w') as f:
    json.dump(date, f)


age = {
  "entity": "Age",
  "ruleScope": "sentence",
   "matchScope":"token",
  "regex":"\\s*(0?[1-9]|[1-9][0-9]|[1][1-9][1-9]|200){1,2}[\\s-,]+|(?i)\\b(?:zero|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)\\b(?=\\s*year)|\\b(?:(?:one|two|three|four|five|six|seven|eight|nine)? hundred(?:\\sand)?\\s)?(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)[\\s-]?)?\\b(?:one|two|three|four|five|six|seven|eight|nine)?(?=\\syear)",
  "prefix":["age of"],
  "suffix": ["-years-old",
             "years-old",
             "-year-old",
             "-months-old",
             "-month-old",
             "-months-old",
             "-day-old",
             "-days-old",
             "month old",
             "days old",
             "year old",
             "years old", 
             "years",
             "year", 
             "months", 
             "old"
              ],
  "contextLength": 25,
  "context": [],
  "contextException": ["ago"],
  "exceptionDistance": 10
}

with open('age.json', 'w') as f:
    json.dump(age, f)

In [4]:
da = DocumentAssembler().setInputCol("text").setOutputCol("document")
sd = SentenceDetector().setInputCols("document").setOutputCol("sentence")
tk = Tokenizer().setInputCols("sentence").setOutputCol("token")

gender_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_gender") \
        .setJsonPath("gender.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)\
        .setDictionary('gender.csv', read_as=ReadAs.TEXT, options={"delimiter":","})
        
age_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_age") \
        .setJsonPath("age.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)
        
date_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_date") \
        .setJsonPath("date.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)

merger1 = ChunkMergeApproach().setInputCols("entity_gender","entity_age").setOutputCol("combined")
merger2 = ChunkMergeApproach().setInputCols("combined","entity_date").setOutputCol("combined")

pl = Pipeline().setStages([da,sd,tk,gender_contextual_parser,age_contextual_parser,date_contextual_parser,merger1,merger2])

In [5]:
merged_data = pl.fit(data_chunk_merge).transform(data_chunk_merge).cache()

In [7]:
merged_data.selectExpr("id","explode(arrays_zip(entity_gender.begin,entity_gender.result, entity_gender.metadata)) as a")\
.selectExpr("id","a['0'] as begin","a['1'] as ner_chunk","a['2'].field as entity")\
.orderBy("id","begin").show(100, False)

+---+-----+---------+------+
|id |begin|ner_chunk|entity|
+---+-----+---------+------+
|1  |14   |female   |Gender|
|1  |471  |she      |Gender|
|1  |562  |she      |Gender|
|1  |668  |she      |Gender|
|1  |835  |her      |Gender|
|1  |1377 |she      |Gender|
|1  |1517 |her      |Gender|
|1  |2097 |her      |Gender|
|1  |2141 |her      |Gender|
|1  |2236 |boy      |Gender|
|1  |2284 |girl     |Gender|
|1  |2360 |girl     |Gender|
+---+-----+---------+------+



In [8]:
merged_data.selectExpr("id","explode(arrays_zip(entity_age.begin,entity_age.result, entity_age.metadata)) as a")\
.selectExpr("id","a['0'] as begin","a['1'] as ner_chunk","a['2'].field as entity")\
.orderBy("id","begin").show(100, False)

+---+-----+-----------+------+
|id |begin|ner_chunk  |entity|
+---+-----+-----------+------+
|1  |2    |28-year-old|Age   |
+---+-----+-----------+------+



In [9]:
merged_data.selectExpr("id","explode(arrays_zip(entity_date.begin,entity_date.result, entity_date.metadata)) as a")\
.selectExpr("id","a['0'] as begin","a['1'] as ner_chunk","a['2'].field as entity")\
.orderBy("id","begin").show(100, False)

+---+-----+----------+------+
|id |begin|ner_chunk |entity|
+---+-----+----------+------+
|1  |2472 |21-02-2020|Date  |
|1  |2484 |21/04/2020|Date  |
+---+-----+----------+------+



In [10]:
merged_data.selectExpr("id","explode(arrays_zip(combined.result, combined.metadata)) as a")\
.selectExpr("id","a['0'] as chunk","a['1'].entity as entity").show(100, False)

+---+-----------+------+
|id |chunk      |entity|
+---+-----------+------+
|1  |28-year-old|Age   |
|1  |female     |Gender|
|1  |she        |Gender|
|1  |she        |Gender|
|1  |she        |Gender|
|1  |her        |Gender|
|1  |she        |Gender|
|1  |her        |Gender|
|1  |her        |Gender|
|1  |her        |Gender|
|1  |boy        |Gender|
|1  |girl       |Gender|
|1  |girl       |Gender|
|1  |21-02-2020 |Date  |
|1  |21/04/2020 |Date  |
+---+-----------+------+

